In [84]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import cross_validation
from sklearn.preprocessing import LabelBinarizer
import sklearn
import pandas as pd
import numpy as np
import xgboost as xg
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


%matplotlib inline

In [61]:
CROSS_VAL_RESULTS_DICT={}
CROSS_VAL_RESULTS_LIST=[]

In [62]:
c_elegans_basic_cleaned_data = pd.read_csv("../results/dummied_c_elegans_30mers.csv")
c_elegans_features = np.load("../data/c_elegans_features.npy")
c_elegans_labels = [label.strip() for label in open("../data/c_elegans_names.txt").readlines()]

In [63]:
df = pd.DataFrame(c_elegans_features, columns=c_elegans_labels)
# df = df.drop(axis = 1, labels = ["Successful Injections", "Success"])
c_elegans_basic_cleaned_data

,Unnamed: 0,Experimenter,sgRNA length,InjectorExperienceLevel,GuideSoftware_No,GuideSoftware_Yes,Success_No,Success_Yes,DNA_DNA,DNA_RNA,...,NGGX_pd.Order2_P5_0.0,NGGX_pd.Order2_P5_1.0,NGGX_pd.Order2_P6_0.0,NGGX_pd.Order2_P6_1.0,NGGX_pd.Order2_P7_0.0,NGGX_pd.Order2_P8_0.0,NGGX_pd.Order2_P8_1.0,NGGX_pd.Order2_P9_0.0,NGGX_pd.Order2_P9_1.0,target
0,0,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.593750
1,1,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.025000
2,2,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.085714
3,3,1,23,3,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.800000
4,4,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.984000
5,5,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.091667
6,6,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.144444
7,7,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.240000
8,8,1,23,2,1.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.702857
9,9,2,22,1,0.0,1.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.050000


In [64]:
dummy_df = pd.get_dummies(df).dropna(axis=0)
dummy_df.shape

(73, 147)

In [65]:
dummy_df_basic_data = c_elegans_basic_cleaned_data
dummy_df_basic_data.shape

(80, 680)

In [66]:
cols = dummy_df.columns.tolist()
# cols.append(cols.pop(cols.index('lfc')))
dummy_df = dummy_df.reindex(columns= cols)

In [67]:
cols_basic = dummy_df_basic_data.columns.tolist()
dummy_df_basic_data = dummy_df_basic_data.reindex(columns=cols_basic)

In [68]:
dummy_df.columns[-1]

'NGGX_pd.Order2_P9'

In [69]:
X, y = dummy_df.iloc[:,:-1].values, dummy_df.iloc[:,-1].values

In [70]:
X2, y2 = dummy_df_basic_data.iloc[:,:-1].values, dummy_df_basic_data.iloc[:,-1].values

In [73]:
basic_model = GradientBoostingRegressor()
basic_cross_val = cross_validation.cross_val_score(estimator=basic_model, X=X2, y=y2, cv = 5, scoring="mean_squared_error")
sum(abs(cross_val))/len(cross_val)

CROSS_VAL_RESULTS_DICT['basic_cross_val'] = basic_cross_val
CROSS_VAL_RESULTS_LIST.append(basic_cross_val)

In [72]:
-sum([-2.51362872, -2.83527368, -2.50070136, -5.03231367, -1.77300729])/5

2.930984944

In [77]:
model = xg.XGBRegressor()
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")

In [78]:
-sum(cross_val)/len(cross_val)

{'XGBRegressor_added_features': array([ -1.98590120e-01,  -6.67651487e-02,  -4.14044412e-02,
         -9.11990240e-05,  -6.73603930e-02]),
 'added_features': array([ -1.98590120e-01,  -6.67651487e-02,  -4.14044412e-02,
         -9.11990240e-05,  -6.73603930e-02]),
 'basic_cross_val': array([-0.13950467, -0.01812591, -0.00076974, -0.00118882, -0.0038929 ])}

In [79]:
2.930984944/2.3380535190836502

1.2536004501508315

In [80]:
model = xg.XGBRegressor(n_estimators=1000)
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
CROSS_VAL_RESULTS_DICT['XGBoost'] = cross_val
CROSS_VAL_RESULTS_LIST.append(cross_val)

In [81]:
-sum(cross_val)/len(cross_val)
CROSS_VAL_RESULTS_DICT

{'XGBRegressor_added_features': array([ -1.98590120e-01,  -6.67651487e-02,  -4.14044412e-02,
         -9.11990240e-05,  -6.73603930e-02]),
 'XGBoost': array([ -1.98591369e-01,  -6.67018889e-02,  -4.12823025e-02,
         -9.06024820e-05,  -6.75319308e-02]),
 'added_features': array([ -1.98590120e-01,  -6.67651487e-02,  -4.14044412e-02,
         -9.11990240e-05,  -6.73603930e-02]),
 'basic_cross_val': array([-0.13950467, -0.01812591, -0.00076974, -0.00118882, -0.0038929 ])}

In [85]:
model = RandomForestRegressor(bootstrap=False, max_features=0.35, min_samples_leaf=1, min_samples_split=8, n_estimators=100)
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
CROSS_VAL_RESULTS_DICT['first iter'] = cross_val
CROSS_VAL_RESULTS_LIST.append(cross_val)

In [86]:
model = DecisionTreeRegressor(max_depth=4, min_samples_leaf=3, min_samples_split=4)
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
CROSS_VAL_RESULTS_DICT['second iter'] = cross_val
CROSS_VAL_RESULTS_LIST.append(cross_val)

In [ ]:
model = make_pipeline(
    FeatureAgglomeration(affinity="l2", linkage="average"),
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=7, p=2, weights="distance")
)
cross_val = cross_validation.cross_val_score(estimator=model, X=X, y=y, cv = 5, scoring="mean_squared_error")
CROSS_VAL_RESULTS_DICT['XGBoost'] = cross_val
CROSS_VAL_RESULTS_LIST.append(cross_val)

In [39]:
[i for i,j in enumerate(dummy_df.columns.values) if j == "lfc"]

[]

In [40]:
! git add baseline.ipynb